# 🍫 [Day 19](https://adventofcode.com/2019/day/19)

In [1]:
def run_program(p, inputs, init_op=0, init_base=0):
    #Inputs are given in reverse order (pop)
    op = init_op
    relative_base = init_base
    last_diagnostic = []
    while p[op] != 99:
        codes = "%05d" % p[op]
        codes = [int(codes[0]), int(codes[1]), int(codes[2]), int(codes[3:])]
        # inputs
        if codes[-1] == 3:
            if not len(inputs):
                # Game is waiting for next move
                break
            assert codes[1] == 0
            target = p[op + 1] + (relative_base if codes[2] == 2 else 0)
            if target >= len(p): 
                p += [0] * (target - len(p) + 1)
            p[target] = inputs.pop()
            op += 2
        # unary ops
        elif codes[-1] in  [4, 9]:
            # read parameter
            assert codes[1] == 0
            param = p[op + 1]
            if (codes[2] % 2) == 0:
                try:
                    param = p[param + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    param = 0
            # output
            if codes[-1] == 4:
                last_diagnostic.append(param)
            # update relative base
            else:
                relative_base += param
            # next instr
            op += 2
        else:
            # read parameters in correct mode
            x, y = p[op + 1:op + 3]
            if (codes[2] % 2) == 0:
                try:
                    x = p[x + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    x = 0
            if not (codes[1] % 2):
                try:
                    y = p[y + (relative_base if codes[1] == 2 else 0)]
                except IndexError:
                    y = 0
            # Read target and alloc ate more memory if needed
            target = p[op + 3] + (relative_base if codes[0] == 2 else 0)
            if target >= len(p): 
                p += [0] * (target - len(p) + 1)
            # addition and multiplication
            if codes[-1] in [1, 2]:
                p[target] = x + y if codes[-1] == 1 else x * y
                op += 4
            # Comparison result
            elif codes[-1] == 7:
                p[target] = int(x < y)
                op += 4
            elif codes[-1] == 8:
                p[target] = int(x == y)
                op += 4
            # Jump if eq
            elif (codes[-1] == 5 and x != 0) or (codes[-1] == 6 and x == 0):
                op = y  
            # Jump instruction that failed their test
            else:
                op += 3
    return last_diagnostic, 0, op, relative_base


import numpy as np
def map_out_beam(program, n=50):
    beam = np.zeros((n, n))
    for x in range(n):
        for y in range(n):
            beam[x, y] = run_program([z for z in program], [x, y])[0][0]
    return int(np.sum(beam))


def find_square(program, s=100, n=1000):
    # Let's try to estimate the line equations delimiting the beam
    x1 = s; x2 = 2 * s
    d1, d2 = [], []
    for x in [x1, x2]:
        start, end = None, None
        for y in range(n):
            o = run_program([z for z in program], [y, x])[0][0]
            if start is None and o == 1:
                start = y
            elif start is not None and end is None and o == 0:
                end = y - 1
                break
        d2.append((x, start))
        d1.append((x, end))
    # lines equations
    a1 = (d1[1][1] - d1[0][1]) / (d1[1][0] - d1[0][0])
    a2 = (d2[1][1] - d2[0][1]) / (d2[1][0] - d2[0][0])
    b1 =  d1[0][1] - a1 * x1
    b2 =  d2[0][1] - a2 * x1
    # (D1) y = a1X + b1
    # (D2) y = a2X + b2
    # We're looking for (x0, y0) such that
    # (X + 100 - 1, Y) is on D2
    # (X, Y + 100 - 1) is on D1
    x0 = (b2 - b1 + s - 1 + a2 * (s - 1)) / (a1 - a2)
    y0 = a2 * (x0 + s - 1) + b2
    return (lambda x: a1 * x + b1, 
            lambda x: a2 * x + b2, 
            int(np.ceil(x0)),
            int(np.ceil(y0)))

In [3]:
with open("inputs/day19.txt", 'r') as f:
    inputs = list(map(int, f.read().split(',')))
    
print("The beam covers {} cells in a 50x50 square".format(map_out_beam(inputs)))

# Part 2: Get coordinates and double ceck
_, _, x0, y0 = find_square(inputs)


print("\n\nSolution of part 2:", x0 * 10000 + y0)
print("X = {}, Y={}".format(x0, y0))
print("In beam ?")
print(run_program([z for z in inputs], [y0, x0 + 99])[0][0])
print(run_program([z for z in inputs], [y0 + 99, x0])[0][0])
print("Outside beam ? ")
print(run_program([z for z in inputs], [y0, x0 + 100])[0][0])
print(run_program([z for z in inputs], [y0 + 100, x0])[0][0])

The beam covers 166 cells in a 50x50 square


Solution of part 2: 3790981
X = 379, Y=981
In beam ?
1
1
Outside beam ? 
0
0
